In [48]:
from collections import namedtuple

Entry = namedtuple('Entry', ['uniques', 'digits'])

with open('day_8_test_input') as f:
    entries = []
    for line in f.readlines():
        uniques_str, _, digits_str = line.strip().partition(' | ')
        uniques = uniques_str.split(' ')
        digits = digits_str.split(' ')
        entries.append(Entry(uniques, digits))

print(entries)

[Entry(uniques=['be', 'cfbegad', 'cbdgef', 'fgaecd', 'cgeb', 'fdcge', 'agebfd', 'fecdb', 'fabcd', 'edb'], digits=['fdgacbe', 'cefdb', 'cefbgd', 'gcbe']), Entry(uniques=['edbfga', 'begcd', 'cbg', 'gc', 'gcadebf', 'fbgde', 'acbgfd', 'abcde', 'gfcbed', 'gfec'], digits=['fcgedb', 'cgb', 'dgebacf', 'gc']), Entry(uniques=['fgaebd', 'cg', 'bdaec', 'gdafb', 'agbcfd', 'gdcbef', 'bgcad', 'gfac', 'gcb', 'cdgabef'], digits=['cg', 'cg', 'fdcagb', 'cbg']), Entry(uniques=['fbegcd', 'cbd', 'adcefb', 'dageb', 'afcb', 'bc', 'aefdc', 'ecdab', 'fgdeca', 'fcdbega'], digits=['efabcd', 'cedba', 'gadfec', 'cb']), Entry(uniques=['aecbfdg', 'fbg', 'gf', 'bafeg', 'dbefa', 'fcge', 'gcbea', 'fcaegb', 'dgceab', 'fcbdga'], digits=['gecf', 'egdcabf', 'bgf', 'bfgea']), Entry(uniques=['fgeab', 'ca', 'afcebg', 'bdacfeg', 'cfaedg', 'gcfdb', 'baec', 'bfadeg', 'bafgc', 'acf'], digits=['gebdcfa', 'ecba', 'ca', 'fadegcb']), Entry(uniques=['dbcfg', 'fgd', 'bdegcaf', 'fgec', 'aegbdf', 'ecdfab', 'fbedc', 'dacgb', 'gdcebf', 'gf'

In [49]:
interesting_lengths = set([2, 3, 4, 7])
num_interesting_digits = 0
for entry in entries:
    num_interesting_digits += len([d for d in entry.digits if len(d) in interesting_lengths])
print(num_interesting_digits)

26


In [50]:
digits_map = {
    'a': [0, 2, 3, 5, 6, 7, 8, 9],
    'b': [0, 4, 5, 6, 8, 9],
    'c': [0, 1, 2, 3, 4, 7, 8, 9],
    'd': [2, 3, 4, 5, 6, 8, 9],
    'e': [0, 2, 6, 8],
    'f': [0, 1, 3, 4, 5, 6, 7, 8, 9],
    'g': [0, 2, 3, 5, 6, 8, 9]
}

solver = {
    'abcefg': 0,
    'cf': 1,
    'acdeg': 2,
    'acdfg': 3,
    'bcdf': 4,
    'abdfg': 5,
    'abdefg': 6,
    'acf': 7,
    'abcdefg': 8,
    'abcdfg': 9
}

def get_counts(entry):
    counts = {}
    for letter in 'abcdefg':
        counts[letter] = 0
    for unique in entry.uniques:
        for letter in unique:
            counts[letter] = counts[letter] + 1
    return counts, {v: k for k, v in counts.items()}

def get_digit_by_size(entry, size):
    choices = [d for d in entry.uniques if len(d) == size]
    if (len(choices) != 1):
        raise Exception('Logic error')
    return choices[0]

def sub(val, minus):
    result = val
    for c in minus:
        result = result.replace(c, '')
    return result

def count_other(val, letters):
    for letter in letters:
        if letter not in val:
            return
    count = 0
    for c in val:
        if c not in letters:
            count += 1
    return count

def get_by_count_other(entry, letters, count):
    result = None
    for unique in entry.uniques:
        if count_other(unique, letters) == count:
            if result is not None:
                raise Exception('Logic error')
            result = unique
    if result is None:
        raise Exception('Logic error')
    return result

def extract(letter_map, letters):
    result = ''
    for letter in letters:
        result += letter_map[letter]
    return result
    
def solve_digit(inv_letter_map, digit):
    mapped = [inv_letter_map[d] for d in digit]
    normalized = ''.join(sorted(mapped))
    return solver[normalized]

def digits_to_num(digits):
    mult = 1
    result = 0
    for i in reversed(range(len(digits))):
        result += digits[i] * mult
        mult *= 10
    return result
    
full_sum = 0

for entry in entries:
    counts, counts_invert = get_counts(entry)

    # f is the only letter in 9 uniques
    letter_map = {}
    letter_map['f'] = counts_invert[9]

    # one is the only digit with 2 wires, since we know f we can find c
    one = get_digit_by_size(entry, 2)
    letter_map['c'] = sub(one, letter_map['f'])

    # seven is the only digit with 3 wires since we know cf we can find a
    seven = get_digit_by_size(entry, 3)
    letter_map['a'] = sub(seven, extract(letter_map, 'cf'))

    # four is the only digit with 4 wires since we know cf we can find
    # bd but not which is is which
    four = get_digit_by_size(entry, 4)
    bd = sub(four, extract(letter_map, 'cf'))

    abcdf = bd + extract(letter_map, 'acf')
    # nine will be the only unique that has abcdf and exactly 1 other wire (g)
    nine = get_by_count_other(entry, abcdf, 1)
    # subtract abcdf to get g
    letter_map['g'] = sub(nine, abcdf)

    # eight is the only digit with 7 wires
    eight = get_digit_by_size(entry, 7)

    # subtract abcdfg to find e
    abcdfg = abcdf + letter_map['g']
    letter_map['e'] = sub(eight, abcdfg)

    acefg = extract(letter_map, 'acefg')

    # zero is the only unique that has acefg and exactly 1 other wire (b)
    zero = get_by_count_other(entry, acefg, 1)
    letter_map['b'] = sub(zero, acefg)

    # now, since we know b, we can take apart bd to find d
    letter_map['d'] = sub(bd, letter_map['b'])

    inv_letter_map = {v: k for k, v in letter_map.items()}
    digits = [solve_digit(inv_letter_map, digit) for digit in entry.digits]
    print(digits)
    full_sum += digits_to_num(digits)
print(full_sum)

[8, 3, 9, 4]
[9, 7, 8, 1]
[1, 1, 9, 7]
[9, 3, 6, 1]
[4, 8, 7, 3]
[8, 4, 1, 8]
[4, 5, 4, 8]
[1, 6, 2, 5]
[8, 7, 1, 7]
[4, 3, 1, 5]
61229
